# 1. install and import dependencise

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2 

In [2]:
# drawing utilities -> mediapipe makes it a whole heap easier to draw the different componets of my model using these drawing utitlies
# 쉽게 말하면 drawing helper
# ex) landmarks
mp_drawing = mp.solutions.drawing_utils


mp_holistic = mp.solutions.holistic

## makes some detections

In [3]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [8]:
results.pose_landmarks.landmark[0]

x: 0.5283795595169067
y: 0.664250910282135
z: -0.8024789094924927
visibility: 0.9998429417610168

#  2. capture landmarks & export to csv

In [16]:
import csv
import os
import numpy as np

In [17]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [18]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [19]:
len(landmarks)

2005

In [20]:
landmarks[-1]

'v501'

In [21]:
with open('coords.csv', mode='w', newline='') as f :
    csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [34]:
class_name = 'happy'

In [35]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # extract landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in face]).flatten())
            
            # concatenate pose and face landmarks
            row = pose_row + face_row
            row.insert(0, class_name)
            
            with open('coords.csv', mode='a', newline='') as f :
                csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass
        
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 3. Train custom model using scikit learn

### 3.1 Read in collected Data and process

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [37]:
pwd

'/Users/richardpark/AISchool/final-project/Just-DDance/model'

In [38]:
df = pd.read_csv('./coords.csv')

In [39]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Victorious,0.599380,0.693755,-0.982394,0.999255,0.604920,0.654998,-0.972673,0.999329,0.612517,...,-0.011710,0.0,0.663894,0.723086,0.014891,0.0,0.668023,0.713335,0.015900,0.0
1,Victorious,0.592535,0.744515,-1.196134,0.999315,0.604460,0.686128,-1.199293,0.999382,0.612738,...,-0.010861,0.0,0.667263,0.728858,0.017163,0.0,0.671485,0.718021,0.018386,0.0
2,Victorious,0.566658,0.757913,-1.199814,0.999372,0.590648,0.701059,-1.210175,0.999434,0.602824,...,-0.008244,0.0,0.667931,0.735280,0.019406,0.0,0.672394,0.725005,0.020619,0.0
3,Victorious,0.572151,0.756316,-1.125300,0.999355,0.589721,0.704806,-1.137349,0.999416,0.601464,...,-0.009371,0.0,0.671657,0.731801,0.018310,0.0,0.676163,0.721182,0.019454,0.0
4,Victorious,0.572051,0.756871,-1.150161,0.999339,0.590155,0.706089,-1.159330,0.999410,0.601602,...,-0.011257,0.0,0.672866,0.732706,0.016223,0.0,0.677801,0.721463,0.017341,0.0


In [40]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1189,happy,0.540642,0.753340,-1.276745,0.998898,0.571450,0.658691,-1.210149,0.998953,0.588971,...,-0.014473,0.0,0.638089,0.616423,0.012381,0.0,0.643917,0.605572,0.013558,0.0
1190,happy,0.540852,0.750458,-1.296617,0.998866,0.571039,0.656088,-1.224671,0.998926,0.588600,...,-0.012356,0.0,0.633472,0.611322,0.014308,0.0,0.639405,0.599619,0.015536,0.0
1191,happy,0.540480,0.748437,-1.249703,0.998905,0.570722,0.654443,-1.173183,0.998962,0.588207,...,-0.013222,0.0,0.627800,0.607132,0.012388,0.0,0.633631,0.595565,0.013553,0.0
1192,happy,0.540266,0.748052,-1.241792,0.998894,0.570305,0.651978,-1.166810,0.998952,0.587856,...,-0.014314,0.0,0.624667,0.607139,0.009799,0.0,0.630818,0.595255,0.010844,0.0
1193,happy,0.540141,0.746629,-1.283992,0.998930,0.569911,0.651158,-1.208311,0.998995,0.587510,...,-0.012668,0.0,0.621243,0.600385,0.012478,0.0,0.627359,0.589637,0.013500,0.0


In [41]:
df[df['class'] == 'Victorious']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Victorious,0.599380,0.693755,-0.982394,0.999255,0.604920,0.654998,-0.972673,0.999329,0.612517,...,-0.011710,0.0,0.663894,0.723086,0.014891,0.0,0.668023,0.713335,0.015900,0.0
1,Victorious,0.592535,0.744515,-1.196134,0.999315,0.604460,0.686128,-1.199293,0.999382,0.612738,...,-0.010861,0.0,0.667263,0.728858,0.017163,0.0,0.671485,0.718021,0.018386,0.0
2,Victorious,0.566658,0.757913,-1.199814,0.999372,0.590648,0.701059,-1.210175,0.999434,0.602824,...,-0.008244,0.0,0.667931,0.735280,0.019406,0.0,0.672394,0.725005,0.020619,0.0
3,Victorious,0.572151,0.756316,-1.125300,0.999355,0.589721,0.704806,-1.137349,0.999416,0.601464,...,-0.009371,0.0,0.671657,0.731801,0.018310,0.0,0.676163,0.721182,0.019454,0.0
4,Victorious,0.572051,0.756871,-1.150161,0.999339,0.590155,0.706089,-1.159330,0.999410,0.601602,...,-0.011257,0.0,0.672866,0.732706,0.016223,0.0,0.677801,0.721463,0.017341,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Victorious,0.510297,0.820866,-0.900196,0.998119,0.532448,0.739896,-0.842356,0.998305,0.549524,...,-0.003742,0.0,0.585997,0.712806,0.016965,0.0,0.590110,0.703885,0.017697,0.0
402,Victorious,0.511063,0.827187,-0.928155,0.997971,0.533094,0.745048,-0.869322,0.998191,0.550052,...,-0.001785,0.0,0.586739,0.709851,0.019526,0.0,0.590936,0.701350,0.020276,0.0
403,Victorious,0.511532,0.829641,-0.955586,0.997839,0.533545,0.747393,-0.895556,0.998097,0.550468,...,-0.001323,0.0,0.586677,0.709343,0.019595,0.0,0.590900,0.700838,0.020335,0.0
404,Victorious,0.511658,0.832464,-0.951546,0.997763,0.533703,0.749783,-0.890136,0.997999,0.550707,...,-0.002518,0.0,0.588510,0.708482,0.018595,0.0,0.592803,0.699878,0.019343,0.0


In [42]:
x = df.drop(['class'], axis=1) # features
y = df['class'] # target value

In [43]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.599380,0.693755,-0.982394,0.999255,0.604920,0.654998,-0.972673,0.999329,0.612517,0.650905,...,-0.011710,0.0,0.663894,0.723086,0.014891,0.0,0.668023,0.713335,0.015900,0.0
1,0.592535,0.744515,-1.196134,0.999315,0.604460,0.686128,-1.199293,0.999382,0.612738,0.681001,...,-0.010861,0.0,0.667263,0.728858,0.017163,0.0,0.671485,0.718021,0.018386,0.0
2,0.566658,0.757913,-1.199814,0.999372,0.590648,0.701059,-1.210175,0.999434,0.602824,0.694663,...,-0.008244,0.0,0.667931,0.735280,0.019406,0.0,0.672394,0.725005,0.020619,0.0
3,0.572151,0.756316,-1.125300,0.999355,0.589721,0.704806,-1.137349,0.999416,0.601464,0.698982,...,-0.009371,0.0,0.671657,0.731801,0.018310,0.0,0.676163,0.721182,0.019454,0.0
4,0.572051,0.756871,-1.150161,0.999339,0.590155,0.706089,-1.159330,0.999410,0.601602,0.700649,...,-0.011257,0.0,0.672866,0.732706,0.016223,0.0,0.677801,0.721463,0.017341,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.540642,0.753340,-1.276745,0.998898,0.571450,0.658691,-1.210149,0.998953,0.588971,0.664172,...,-0.014473,0.0,0.638089,0.616423,0.012381,0.0,0.643917,0.605572,0.013558,0.0
1190,0.540852,0.750458,-1.296617,0.998866,0.571039,0.656088,-1.224671,0.998926,0.588600,0.661492,...,-0.012356,0.0,0.633472,0.611322,0.014308,0.0,0.639405,0.599619,0.015536,0.0
1191,0.540480,0.748437,-1.249703,0.998905,0.570722,0.654443,-1.173183,0.998962,0.588207,0.659905,...,-0.013222,0.0,0.627800,0.607132,0.012388,0.0,0.633631,0.595565,0.013553,0.0
1192,0.540266,0.748052,-1.241792,0.998894,0.570305,0.651978,-1.166810,0.998952,0.587856,0.657418,...,-0.014314,0.0,0.624667,0.607139,0.009799,0.0,0.630818,0.595255,0.010844,0.0


In [44]:
y

0       Victorious
1       Victorious
2       Victorious
3       Victorious
4       Victorious
           ...    
1189         happy
1190         happy
1191         happy
1192         happy
1193         happy
Name: class, Length: 1194, dtype: object

In [45]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [46]:
display(X_train.shape)
display(y_train.shape)

display(X_test.shape)
display(y_test.shape)

(955, 2004)

(955,)

(239, 2004)

(239,)

### 3.2 Train machine learning classification model

In [47]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from tqdm import tqdm

In [48]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [49]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [50]:
fit_models = {}

for algo, pipeline in tqdm(pipelines.items()):
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

  0%|          | 0/4 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|██████████| 4/4 [02:14<00:00, 33.62s/it]


In [51]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [52]:
fit_models['rc'].predict(X_test)

array(['sad', 'sad', 'sad', 'Victorious', 'Victorious', 'Victorious',
       'wakanda forever', 'wakanda forever', 'sad', 'wakanda forever',
       'wakanda forever', 'Victorious', 'Victorious', 'wakanda forever',
       'wakanda forever', 'wakanda forever', 'wakanda forever', 'sad',
       'wakanda forever', 'wakanda forever', 'happy', 'Victorious',
       'wakanda forever', 'sad', 'sad', 'happy', 'Victorious',
       'wakanda forever', 'wakanda forever', 'happy', 'wakanda forever',
       'Victorious', 'Victorious', 'sad', 'sad', 'Victorious', 'sad',
       'Victorious', 'wakanda forever', 'happy', 'happy', 'Victorious',
       'sad', 'Victorious', 'sad', 'sad', 'sad', 'Victorious',
       'Victorious', 'wakanda forever', 'Victorious', 'sad', 'happy',
       'Victorious', 'happy', 'sad', 'happy', 'wakanda forever', 'happy',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'happy',
       'happy', 'happy', 'Victorious', 'sad', 'Victorious',
       'wakanda forever', 'sad

### 3.3 Evaluate and Serialize model

In [53]:
from sklearn.metrics import accuracy_score # calculate accuracy
import pickle # 

In [54]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rf 0.9916317991631799
rc 0.9916317991631799
gb 0.99581589958159


In [55]:
fit_models['rc'].predict(X_test)

array(['sad', 'sad', 'sad', 'Victorious', 'Victorious', 'Victorious',
       'wakanda forever', 'wakanda forever', 'sad', 'wakanda forever',
       'wakanda forever', 'Victorious', 'Victorious', 'wakanda forever',
       'wakanda forever', 'wakanda forever', 'wakanda forever', 'sad',
       'wakanda forever', 'wakanda forever', 'happy', 'Victorious',
       'wakanda forever', 'sad', 'sad', 'happy', 'Victorious',
       'wakanda forever', 'wakanda forever', 'happy', 'wakanda forever',
       'Victorious', 'Victorious', 'sad', 'sad', 'Victorious', 'sad',
       'Victorious', 'wakanda forever', 'happy', 'happy', 'Victorious',
       'sad', 'Victorious', 'sad', 'sad', 'sad', 'Victorious',
       'Victorious', 'wakanda forever', 'Victorious', 'sad', 'happy',
       'Victorious', 'happy', 'sad', 'happy', 'wakanda forever', 'happy',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'happy',
       'happy', 'happy', 'Victorious', 'sad', 'Victorious',
       'wakanda forever', 'sad

In [56]:
y_test

533                 sad
618                 sad
671                 sad
220          Victorious
58           Victorious
             ...       
265          Victorious
864     wakanda forever
328          Victorious
1026              happy
76           Victorious
Name: class, Length: 239, dtype: object

In [57]:
with open('richard_model1.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [58]:
ls

coords.csv
facial_expression_model_structure.json
facial_expression_model_weights.h5
richard_model1.pkl
richard_trained_model_FaceMotionDetection.pkl


In [9]:
pwd

'/Users/richardpark/AISchool/final-project/Just-DDance/model'

In [59]:
with open('/Users/richardpark/AISchool/final-project/Just-DDance/model/richard_model1.pkl', 'rb') as f:
    model = pickle.load(f)

In [60]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [62]:
# feed from webcam - connecting to webcam 
cap = cv2.VideoCapture(0)

cv2.startWindowThread()
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened(): # check wether or not webcam is open
        ret, frame = cap.read() # frame 변수에 webcam 으로 부터 받아온 이미지를 저장한다.
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections - holistic model에 detection 하는 process 함수에 image 전달
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # extract landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in face]).flatten())
            
            # concatenate pose and face landmarks
            row = pose_row + face_row
            
            # row.insert(0, class_name)
            
            # with open('coords.csv', mode='a', newline='') as f :
            #     csv_writer = csv.writer(f, delimiter=',' , quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid f

-1

: 